In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
%matplotlib inline

In [ ]:
dataset=pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')

In [ ]:
dataset

In [ ]:
sex = dataset['sex'].unique()
d1 = dataset[dataset['sex']==sex[0]][['age','DEATH_EVENT']].rename(columns={'DEATH_EVENT':'Died'}).groupby('age').sum().reset_index()
d2 = dataset[dataset['sex']==sex[1]][['age','DEATH_EVENT']].rename(columns={'DEATH_EVENT':'Dont died'}).groupby('age').sum().reset_index()
d1.merge(d2,on='age').set_index('age').style.background_gradient('Blues')

In [ ]:
plt.figure(figsize=(16,9))
sns.displot(data=dataset[dataset['DEATH_EVENT']==1],y='age',element="step",multiple="stack",kde=True)
plt.show()

Here, we can see that most people who died had the ages between 60 and 70 years old

In [ ]:
sns.displot(data=dataset,x='age',hue='DEATH_EVENT',element="step",multiple="stack",kde=True)

In [ ]:
sns.violinplot(x='DEATH_EVENT', y='age', data=dataset)

In [ ]:
plt.figure(figsize=(12,9))
g=sns.displot(data=dataset[dataset['DEATH_EVENT']==1], x="age", col='sex',hue='smoking', multiple="stack",element="step",kde=True)

How we can see, you can't say that smooking (by the dataset) has a lot of correlacion with heart failure.

In [ ]:
sns.violinplot(x='DEATH_EVENT', y='age', data=dataset)

In [ ]:
corr_data = dataset.corr()

mask = np.zeros_like(corr_data)

mask[np.triu_indices_from(mask)] = True

plt.figure(figsize=(20,12))

sns.heatmap(corr_data.abs(),cmap='YlOrBr', annot=True, fmt='.2f',square=True,mask=mask, vmax=.3,linewidths=.5)
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
x_treino,x_teste,y_treino,y_teste=train_test_split(dataset.drop('DEATH_EVENT',axis=1),dataset['DEATH_EVENT'],test_size=0.2,random_state=106)

In [ ]:
print('The training dataset has {} patients, the test has {} patients.'.format(x_treino.shape[0], x_teste.shape[0]))

In [ ]:
modelo_log=LogisticRegression(max_iter=1500,solver='liblinear')
modelo_log.fit(x_treino,y_treino)
predicao_log=modelo_log.predict(x_teste)

In [ ]:
print(classification_report(y_teste,predicao_log))
print("\n")

In [ ]:
acc_logReg = round(modelo_log.score(x_treino,y_treino) * 100, 2)
print("Accuracy of the Logistic Regression model: {}".format(acc_logReg))

In [ ]:
scaler=StandardScaler()
x_treino=scaler.fit_transform(x_treino)
x_teste=scaler.transform(x_teste)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
modelo_randon = RandomForestClassifier(n_estimators=100)
modelo_randon.fit(x_treino,y_treino)
predicao_randon=modelo_randon.predict(x_teste)


In [ ]:
print(classification_report(y_teste,predicao_randon))
print("\n")

In [ ]:
from sklearn.model_selection import cross_val_score
SEED = 50
np.random.seed(SEED)
cv = StratifiedKFold(n_splits = 5, shuffle = True)
results_tree = cross_val_score(modelo_randon, x_treino, y_treino, cv=cv, scoring = 'accuracy')
results_log = cross_val_score(modelo_log, x_treino, y_treino, cv=cv, scoring = 'accuracy')
def intervalo(results):
    mean = results.mean()
    dv = results.std()
    print('Average accuracy: {:.2f}%'.format(mean*100))
    print('Accuracy range: [{:.2f}% ~ {:.2f}%]'
           .format((mean - 2*dv)*100, (mean + 2*dv)*100))
    
print("RandomForestClassifier")
intervalo(results_tree)
print("\n")
print("LogisticRegression")
intervalo(results_log)